# HW4: Graph Neural Networks

* There are two datasets in the `data` folder: `train.pt`, `test.pt`. You will train a GCNN on the train dataset, then make predictions on the test dataset.
* There are two parts in this notebook. `Part I` gives a custom `Dataset` object and loads the datasets. The `QM_Dataset` object inherites from torch geometric `Dataset` object. `Part II` is an example solution.
* This HW is implemented with [Pytorch Geometric (PyG)](https://pytorch-geometric.readthedocs.io/en/latest/index.html). Another popular library for implementing GNNs is [Deep Graph Library (DGL)](https://www.dgl.ai/)

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Sequential
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader

In [3]:
# only on google colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part I. The training and testing dataset are provided

- The train and test datasets were pre-processed graphs. The train dataset contains 20,000 graphs, while the test dataset contains 2,000 graphs.
- Each graph contains the following components:

    - `x`, the matrix containing node features, `[num_of_nodes, num_node_features=11]`
    - `edge_index`, the matrix containing connection information about different nodes, `[2, num_of_edges]`
    - `y`, the label for the graph, `scaler`. The value is set to `0` in the test dataset
    - `pos`, the matrix containing the node positions, `[num_of_nodes, 3]`
    - `edge_attr`, the matrix containing the edge information, `[num_edges, 4]`
    - `names`, index for the graph. For example, `gdb_59377`

- Depending on the graph convolutional layer that is used, different components are needed. For the most basic application, `x`, `edge_index` and `y` will be used.


### Data Augmentation

In [ ]:
from torch_geometric.transforms import BaseTransform
import torch

class RandomEdgeDrop(BaseTransform):
    def __init__(self, drop_prob=0.1):
        self.drop_prob = drop_prob

    def __call__(self, data):
        # Drop edges randomly
        edge_mask = torch.rand(data.edge_index.size(1)) > self.drop_prob
        data.edge_index = data.edge_index[:, edge_mask]
        if data.edge_attr is not None:
            data.edge_attr = data.edge_attr[edge_mask]
        return data

class FeatureNoise(BaseTransform):
    def __init__(self, noise_level=0.1):
        self.noise_level = noise_level

    def __call__(self, data):
        noise = torch.randn(data.x.size()) * self.noise_level
        data.x += noise
        return data

# Compose multiple transformations
from torch_geometric.transforms import Compose
transform = Compose([
    RandomEdgeDrop(drop_prob=0.1),
    FeatureNoise(noise_level=0.05)
])

class QM_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super().__init__()
        self.data = torch.load(path)
        self.transform = transform

    def len(self):
        return len(self.data)

    def get(self, idx):
        data = self.data[idx]
        if self.transform:
            data = self.transform(data)
        return data

train_path = "/content/drive/MyDrive/data/train.pt"
test_path = "/content/drive/MyDrive/data/test.pt"

train_data_ = QM_Dataset(train_path, transform=transform)

# train dataset can be split for validation purposes
train_data, validate_data = torch.utils.data.random_split(train_data_, [19000, 1000])
test_data = QM_Dataset(test_path)

### Without Augmentation

In [46]:
class QM_Dataset(Dataset):
    def __init__(self, path):
        super().__init__(root=".")
        self.data = torch.load(path)

    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]

train_path = "/content/drive/MyDrive/data/train.pt"
test_path = "/content/drive/MyDrive/data/test.pt"

train_data_ = QM_Dataset(train_path)

# train dataset can be split for validation purposes
train_data, validate_data = torch.utils.data.random_split(train_data_, [19000, 1000])
test_data = QM_Dataset(test_path)

## Part II. Example solution

### Net

In [ ]:

from torch_geometric.nn import NNConv, Set2Set

class Net(torch.nn.Module):
    def __init__(self, num_features=11, dim=64):
        super().__init__()
        self.lin0 = torch.nn.Linear(num_features, dim)
        nn = Sequential(Linear(4, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        for _ in range(3):
            out = F.relu(self.conv(out, data.edge_index, data.edge_attr))
        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        return out.view(-1)

### LEC GAT

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import LEConv, GATConv, global_mean_pool

class CombinedGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(CombinedGNN, self).__init__()
        self.conv1 = LEConv(in_channels, hidden_channels, aggr='max')  # Using max aggregation for the LEConv layer
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=heads, concat=True)  # Using multiple attention heads in GAT

        # Dynamically adjust in_channels for the output layer based on the number of heads
        final_in_channels = hidden_channels * heads if heads > 1 else hidden_channels
        self.fc = nn.Linear(final_in_channels, out_channels)  # Linear layer for output

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Process through the first convolutional layer
        x = F.relu(self.conv1(x, edge_index))

        # Process through the second convolutional layer
        x = F.relu(self.conv2(x, edge_index))

        # Pooling over all nodes in each graph to a single vector
        x = global_mean_pool(x, batch)

        # Output layer
        x = self.fc(x)
        x = x.squeeze(-1)

        return x

# Device configuration for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model instantiation and moving to the appropriate device
model = CombinedGNN(in_channels=11, hidden_channels=32, out_channels=1, heads=8).to(device)


In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import LEConv, GATConv, GCNConv, global_mean_pool

class CombinedGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(CombinedGNN, self).__init__()
        self.conv1 = LEConv(in_channels, hidden_channels, aggr='max')
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=heads, concat=True)
        self.conv3 = GCNConv(hidden_channels * heads, hidden_channels)  # Adjusted to correct dimension
        self.fc = nn.Linear(hidden_channels, out_channels)
        self.leaky_relu = nn.LeakyReLU(0.01)  # LeakyReLU with a small negative slope

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.leaky_relu(self.conv1(x, edge_index))
        x = self.leaky_relu(self.conv2(x, edge_index))
        x = self.leaky_relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        x = x.view(-1, 1)  # Reshape output to match the target's shape [batch_size, 1]
        return x

# Model initialization and to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CombinedGNN(in_channels=11, hidden_channels=32, out_channels=1, heads=8).to(device)


In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import LEConv, GATConv, global_mean_pool, BatchNorm

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class LEConvNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(LEConvNet, self).__init__()
        self.conv1 = LEConv(in_channels, hidden_channels, aggr='max')
        self.bn1 = BatchNorm(hidden_channels)
        self.conv2 = LEConv(hidden_channels, hidden_channels, aggr='max')
        self.bn2 = BatchNorm(hidden_channels)
        self.gat = GATConv(hidden_channels, hidden_channels, heads=heads, concat=True)  # Graph Attention Layer
        # Update in_channels for the final layer if concat is True for GAT
        final_in_channels = hidden_channels * heads if heads > 1 else hidden_channels
        self.fc = nn.Linear(final_in_channels, out_channels)
        self.dropout = nn.Dropout(0.5)
        self.swish = Swish()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.swish(self.bn1(self.conv1(x, edge_index)))
        x = self.swish(self.bn2(self.conv2(x, edge_index)))
        x = self.swish(self.gat(x, edge_index))  # Applying GAT layer with Swish activation
        x = self.dropout(x)

        x = global_mean_pool(x, batch)  # Global mean pooling

        x = self.fc(x)
        return x

# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LEConvNet(in_channels=11, hidden_channels=64, out_channels=1, heads=8).to(device)


### GCN 2

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class BaseGCN(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(BaseGCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.pool = global_mean_pool

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.pool(x, batch)
        return x

class StackedModel(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(StackedModel, self).__init__()
        self.gcn = BaseGCN(num_features, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)  # Adapted to single GCN output

    def forward(self, data):
        x = self.gcn(data)
        x = F.relu(x)
        x = self.fc(x)
        return x

# Assuming train_data is available and has the attribute `num_node_features`
model = StackedModel(num_features=train_data[0].num_node_features, hidden_channels=64)


### GNC3

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

class BaseGCN(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(BaseGCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm(hidden_channels)
        self.pool = global_mean_pool
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = self.pool(x, batch)
        return x

class StackedModel(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(StackedModel, self).__init__()
        self.gcn = BaseGCN(num_features, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)  # Adapted to single GCN output
        self.final_act = nn.Sigmoid()  # For binary classification, use Sigmoid. Change as needed.

    def forward(self, data):
        x = self.gcn(data)
        x = F.relu(x)
        x = self.fc(x)
        x = self.final_act(x)  # Use sigmoid for binary classification tasks
        return x

# Assuming train_data is available and has the attribute `num_node_features`
model = StackedModel(num_features=train_data[0].num_node_features, hidden_channels=64)


### GCN + MLP

In [5]:
#!pip install torch torchvision torchaudio
#!pip install torch-geometric

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.pool = global_mean_pool  # Pooling to get graph-level representation

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.pool(x, batch)  # Pool node features to one graph-level feature
        return x

class MLP(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 50)
        self.fc2 = torch.nn.Linear(50, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.view(-1, 1)  # Reshape output to match target shape


class StackedModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, output_dim):
        super(StackedModel, self).__init__()
        self.gcn = GCN(num_features, hidden_channels)
        self.mlp = MLP(hidden_channels, output_dim)

    def forward(self, data):
        graph_features = self.gcn(data)
        output = self.mlp(graph_features)
        return output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = StackedModel(num_features=11, hidden_channels=64, output_dim=1).to(device)


### Train & Eval

In [48]:
import torch
import os
from torch.nn.functional import mse_loss
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
from torch import nn

def train(loader, model, optimizer, criterion, device):
    model.train()  # Set the model to training mode
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)


from sklearn.metrics import mean_absolute_error
import numpy as np

def evaluate(loader, model, criterion, device):
    model.eval()
    total_loss = 0.0
    num_samples = 0  # We'll count the actual number of samples processed
    predictions = []
    actuals = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            loss = criterion(output, data.y)
            total_loss += loss.item() * data.y.size(0)  # Assuming each y corresponds to one sample
            num_samples += data.y.size(0)  # Accumulate the actual number of samples

            predictions.append(output.view(-1).cpu())
            actuals.append(data.y.view(-1).cpu())

    predictions = torch.cat(predictions)
    actuals = torch.cat(actuals)
    mae = mean_absolute_error(actuals.numpy(), predictions.numpy())
    average_loss = total_loss / num_samples  # Compute average loss based on actual samples

    return average_loss, mae


In [49]:
import torch
import os

def train_model(model, train_loader, validate_loader, criterion, optimizer, epochs=100, device=torch.device('cpu')):
    model_dir = '/content/drive/MyDrive/data'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_path = os.path.join(model_dir, 'best10.pth')
    best_val_loss = float('inf')

    for epoch in range(epochs):
        train_loss = train(train_loader, model, optimizer, criterion, device)
        val_loss, val_mae = evaluate(validate_loader, model, criterion, device)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Validation MAE: {val_mae:.4f}')

        if val_mae < best_val_loss:
            best_val_loss = val_mae
            torch.save(model.state_dict(), model_path)
            print(f'Model saved at {model_path} with validation MAE: {best_val_loss:.4f}')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''model_path = '/content/drive/MyDrive/data/best_model9.pth'
model.load_state_dict(torch.load(model_path, map_location=device)) '''


"model_path = '/content/drive/MyDrive/data/best_model9.pth'\nmodel.load_state_dict(torch.load(model_path, map_location=device)) "

In [50]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_data, batch_size=128)
validate_loader = DataLoader(validate_data, batch_size=128)
test_loader = DataLoader(test_data, batch_size=8)

model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99) #BAD
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.1, eps=1e-06) #BAD

train_model(model, train_loader, validate_loader, criterion, optimizer, epochs=1000, device=device)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([104])) that is different to the input size (torch.Size([104, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/1000, Train Loss: 2.0517, Val Loss: 3.4087, Validation MAE: 0.9157
Model saved at /content/drive/MyDrive/data/best10.pth with validation MAE: 0.9157


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([104])) that is different to the input size (torch.Size([104, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2/1000, Train Loss: 1.5778, Val Loss: 3.4277, Validation MAE: 0.8880
Model saved at /content/drive/MyDrive/data/best10.pth with validation MAE: 0.8880


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([104])) that is different to the input size (torch.Size([104, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([104])) that is different to the input size (torch.Size([104, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 103/1000, Train Loss: 0.7033, Val Loss: 4.3158, Validation MAE: 0.6951


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([104])) that is different to the input size (torch.Size([104, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 104/1000, Train Loss: 0.7104, Val Loss: 4.3157, Validation MAE: 0.6975


KeyboardInterrupt: 

In [70]:
model_path = '/content/drive/MyDrive/data/best6.pth'
#model = GCN1().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

## Prediction

In [79]:
model_path = '/content/drive/MyDrive/data/best12.pth'
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [80]:
import pandas as pd

model.eval()
y_pred = []
Idx = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = model(data)
        y_pred.extend(output.squeeze(1).cpu().numpy())  # using squeeze(1) to remove single-dimensional entries from shape
        Idx.extend(data.name)
df = pd.DataFrame({"Idx": Idx, "labels": y_pred})


In [56]:
# upload solution
df.columns = ['Idx', 'labels']
df.to_csv("/content/drive/MyDrive/data/6041.csv", index=False)